# 4 - Influential Classification Models (and Tools)

## Introduction

This Notebook (2) will continue on from the previous sections. This notebook will go through the process of building the __ResNet__ from scratch. 

## Dataset:

For this part of the project, the CIFAR-100 dataset will be used, it is a collection of of 60,000 32x32 images that have 100 classes. CIFAR-100 was originally collected by Alex Krizhevsky, Vinod Nair, and Geoffrey Hinton. It iss also a subset of the 80 million tiny images dataset. There are 500 training images and 100 testing images per class. The 100 classes in the CIFAR-100 are grouped into 20 superclasses.

Source: https://www.cs.toronto.edu/~kriz/cifar.html

Further, the TensorFlow team offers a python package called "tensorflow_datasets" that provides the helper function to download tthis dataset as well as other more common ones. For the purposes of this project, the CIFAR-100 dataset will be download with this package.

Source: https://www.tensorflow.org/datasets/catalog/cifar100

## Requirements:
- Tensorflow 2.0 (GPU is better)
- Tensorflow-Hub
- Keras (GPU is better)

### Import the required libraries:

In [1]:
%matplotlib inline

import tensorflow as tf
import numpy as np
import matplotlib
from matplotlib import pyplot as plt
import timeit

In [2]:
import os
from IPython.display import display, Image
import matplotlib.pyplot as plt

# %matplotlib inline

# Set up the working directory for the images:
image_folderName = 'Description Images'
image_path = os.path.abspath(image_folderName) + '/'

In [3]:
# Set the random set seed number: for reproducibility.
Seed_nb = 42

### GPU Information:

In [4]:
sess = tf.compat.v1.Session(config=tf.compat.v1.ConfigProto(log_device_placement=True))
devices = sess.list_devices()
devices

Device mapping:
/job:localhost/replica:0/task:0/device:GPU:0 -> device: 0, name: GeForce RTX 2070 SUPER, pci bus id: 0000:09:00.0, compute capability: 7.5



[_DeviceAttributes(/job:localhost/replica:0/task:0/device:CPU:0, CPU, 268435456, 9372556792854543533),
 _DeviceAttributes(/job:localhost/replica:0/task:0/device:GPU:0, GPU, 6588305899, 5947880554449152520)]

## 1 - Data Preparation:

To use the TensorFlow package, please ensure to install it: "pip install tensorflow-datasets" or use the Anaconda Navigator.

## 1.1 - Download the Dataset:

In [5]:
import tensorflow_datasets as tfds

In [6]:
# Show the different types of datasets available:
tfds.list_builders()

['abstract_reasoning',
 'aflw2k3d',
 'amazon_us_reviews',
 'bair_robot_pushing_small',
 'bigearthnet',
 'binarized_mnist',
 'binary_alpha_digits',
 'caltech101',
 'caltech_birds2010',
 'caltech_birds2011',
 'cats_vs_dogs',
 'celeb_a',
 'celeb_a_hq',
 'chexpert',
 'cifar10',
 'cifar100',
 'cifar10_corrupted',
 'clevr',
 'cnn_dailymail',
 'coco',
 'coco2014',
 'coil100',
 'colorectal_histology',
 'colorectal_histology_large',
 'curated_breast_imaging_ddsm',
 'cycle_gan',
 'deep_weeds',
 'definite_pronoun_resolution',
 'diabetic_retinopathy_detection',
 'downsampled_imagenet',
 'dsprites',
 'dtd',
 'dummy_dataset_shared_generator',
 'dummy_mnist',
 'emnist',
 'eurosat',
 'fashion_mnist',
 'flores',
 'food101',
 'gap',
 'glue',
 'groove',
 'higgs',
 'horses_or_humans',
 'image_label_folder',
 'imagenet2012',
 'imagenet2012_corrupted',
 'imdb_reviews',
 'iris',
 'kitti',
 'kmnist',
 'lfw',
 'lm1b',
 'lsun',
 'mnist',
 'mnist_corrupted',
 'moving_mnist',
 'multi_nli',
 'nsynth',
 'omniglot',

In [7]:
import urllib3
urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)

In [8]:
# Select the CIFAR-100 dataset:
cifar_builder = tfds.builder("cifar100")
cifar_builder.download_and_prepare()

print(cifar_builder.info)

tfds.core.DatasetInfo(
    name='cifar100',
    version=1.3.1,
    description='This dataset is just like the CIFAR-10, except it has 100 classes containing 600 images each. There are 500 training images and 100 testing images per class. The 100 classes in the CIFAR-100 are grouped into 20 superclasses. Each image comes with a "fine" label (the class to which it belongs) and a "coarse" label (the superclass to which it belongs).',
    urls=['https://www.cs.toronto.edu/~kriz/cifar.html'],
    features=FeaturesDict({
        'coarse_label': ClassLabel(shape=(), dtype=tf.int64, num_classes=20),
        'image': Image(shape=(32, 32, 3), dtype=tf.uint8),
        'label': ClassLabel(shape=(), dtype=tf.int64, num_classes=100),
    }),
    total_num_examples=60000,
    splits={
        'test': 10000,
        'train': 50000,
    },
    supervised_keys=('image', 'label'),
    citation="""@TECHREPORT{Krizhevsky09learningmultiple,
        author = {Alex Krizhevsky},
        title = {Learning multi

From the above printed text, it also shows some useful information such as the number of training and testing samples, the key-names and so on.

## 1.2 - Examine the class labels of the dataset:

In [9]:
# Check out the labels:
print(cifar_builder.info.features["label"].names)

print('The number of classes are: {}'.format(len(cifar_builder.info.features["label"].names)))

['apple', 'aquarium_fish', 'baby', 'bear', 'beaver', 'bed', 'bee', 'beetle', 'bicycle', 'bottle', 'bowl', 'boy', 'bridge', 'bus', 'butterfly', 'camel', 'can', 'castle', 'caterpillar', 'cattle', 'chair', 'chimpanzee', 'clock', 'cloud', 'cockroach', 'couch', 'crab', 'crocodile', 'cup', 'dinosaur', 'dolphin', 'elephant', 'flatfish', 'forest', 'fox', 'girl', 'hamster', 'house', 'kangaroo', 'keyboard', 'lamp', 'lawn_mower', 'leopard', 'lion', 'lizard', 'lobster', 'man', 'maple_tree', 'motorcycle', 'mountain', 'mouse', 'mushroom', 'oak_tree', 'orange', 'orchid', 'otter', 'palm_tree', 'pear', 'pickup_truck', 'pine_tree', 'plain', 'plate', 'poppy', 'porcupine', 'possum', 'rabbit', 'raccoon', 'ray', 'road', 'rocket', 'rose', 'sea', 'seal', 'shark', 'shrew', 'skunk', 'skyscraper', 'snail', 'snake', 'spider', 'squirrel', 'streetcar', 'sunflower', 'sweet_pepper', 'table', 'tank', 'telephone', 'television', 'tiger', 'tractor', 'train', 'trout', 'tulip', 'turtle', 'wardrobe', 'whale', 'willow_tree',

In [10]:
# Check out the coarse labels:
print(cifar_builder.info.features["coarse_label"].names)

print('The number of classes are: {}'.format(len(cifar_builder.info.features["coarse_label"].names)))

['aquatic_mammals', 'fish', 'flowers', 'food_containers', 'fruit_and_vegetables', 'household_electrical_devices', 'household_furniture', 'insects', 'large_carnivores', 'large_man-made_outdoor_things', 'large_natural_outdoor_scenes', 'large_omnivores_and_herbivores', 'medium_mammals', 'non-insect_invertebrates', 'people', 'reptiles', 'small_mammals', 'trees', 'vehicles_1', 'vehicles_2']
The number of classes are: 20


## 1.3 - Define the Input Pipeline:

As the dataset have been downloaded with the help of the package, this section will then define the input pipeline for the models to be trained. These are variables that will be used later on.

In [11]:
import math

In [12]:
# Define the Hyperparameters:
input_shape = [224, 224, 3]
batch_size = 512
nb_epochs = 300

In [13]:
# Divide the dataset into training and validation sets:
train_set_cifar = cifar_builder.as_dataset(split = tfds.Split.TRAIN)
test_set_cifar = cifar_builder.as_dataset(split = tfds.Split.TEST)

In [14]:
# Set the number of classes:
nb_classes = cifar_builder.info.features['label'].num_classes

In [15]:
# Define the number of images for training and validation:
nb_train_imgs = cifar_builder.info.splits['train'].num_examples
nb_valid_imgs = cifar_builder.info.splits['test'].num_examples

## 1.4 - Check out the dataset instances:

This should provide some additional details about the dataset, such as its dtype and shape.

In [16]:
print("The Training set instance: {}".format(train_set_cifar))

The Training set instance: <_OptionsDataset shapes: {coarse_label: (), image: (32, 32, 3), label: ()}, types: {coarse_label: tf.int64, image: tf.uint8, label: tf.int64}>


In [17]:
print("The Testing set instance: {}".format(test_set_cifar))

The Testing set instance: <_OptionsDataset shapes: {coarse_label: (), image: (32, 32, 3), label: ()}, types: {coarse_label: tf.int64, image: tf.uint8, label: tf.int64}>


## 2 - Data Preprocessing:

## 2.1 - Shuffling the dataset:

The notion behind shuffling the dataset before being used for training is so that the model is able to learn from majority of the data in the training set, where unshuffled, the training data can be in the format order of A-R while the testing set is S-Z, so during training the model only learns from what is available from A-R but will later perform badly in the unseen testing data of S-Z. Shuffling, as mentioned, allows the model to train in a more robust manner resulting in a better generalised model.

In [18]:
# Shuffle the Training Dataset: 10,000 times of reshuffling.
train_set_cifar = train_set_cifar.repeat(nb_epochs).shuffle(10000)

## 2.2 - Set up the dataset for compatibility with Keras API (non-estimator) and Image Augmentation:

It should be noted that Tensorflow-Dataset returns batches as feature dictionaries, which is expected by Estimators. Therefore, to train with Keras models, it is better to return the batch content as tuples. A user defined funtion will be needed to do this.

In [19]:
def _prepare_data_func(features, input_shape, augment=False):
    """ This builds a pre-processing function to resize the image into the expected dimenions and allows for an
        optional transformation of the images such as augmentations.
    Parameters:
        - features, is the input data.
        - input_shape, is the expected shape for model by resizing.
        - augment, is a Flag to apply random transformations to the input images.
    Returns:
        - returns Augmented images, Labels
    """
    
    # Convert the input images into tensors:
    input_shape = tf.convert_to_tensor(input_shape)
    
    # Convert TF-dataset feature dictionaries into Tuples:
    image = features['image']
    label = features['label']
    
    # Convert the image data type to "float32" and normalise the data to a range between "0 and 1":
    image = tf.image.convert_image_dtype(image, tf.float32)
    
    # Data Image Augmentation:
    if augment:
        # Apply random horizontal flip::
        image = tf.image.random_flip_left_right(image)
        
        # Apply Brightness and Saturation changes:
        image = tf.image.random_brightness(image, max_delta = 0.1)
        image = tf.image.random_saturation(image, lower = 0.5, upper = 1.5)
        image = tf.clip_by_value(image, 0.0, 1.0) # this keeps the pixel values in check.
        
        # Apply random resizing and cropping back to expected image sizes:
        random_scale_factor = tf.random.uniform([1], minval = 1., maxval = 1.4, dtype = tf.float32)
        
        scaled_height = tf.cast(tf.cast(input_shape[0], tf.float32) * random_scale_factor, tf.int32)
        scaled_width = tf.cast(tf.cast(input_shape[1], tf.float32) * random_scale_factor, tf.int32)
        scaled_shape = tf.squeeze(tf.stack([scaled_height, scaled_width]))
        
        image = tf.image.resize(image, scaled_shape)
        image = tf.image.random_crop(image, input_shape)
    else:
        image = tf.image.resize(image, input_shape[:2])
        
    return image, label

#### Apply the function above on the training and validation set:

NOTE: functools.partial example -> https://stackoverflow.com/questions/15331726/how-does-functools-partial-do-what-it-does

In [20]:
import functools

##### For Training set:

In [21]:
# Set the function for the training data preparation:
prepare_data_func_for_trainSet = functools.partial(_prepare_data_func, input_shape = input_shape, augment = True)

# Map the function to the dataset:
train_set_cifar = train_set_cifar.map(prepare_data_func_for_trainSet, num_parallel_calls = tf.data.experimental.AUTOTUNE)

# Split the training set into batched samples:
train_set_cifar = train_set_cifar.batch(batch_size)

# Set to prefetch the data: improves the perforamnce.
# his allows later elements to be prepared while the current element is being processed. 
# This often improves latency and throughput, at the cost of using additional memory to store prefetched elements.
train_set_cifar= train_set_cifar.prefetch(1)


##### For Validation set: no augment

In [22]:
# Set the function for the Validation data preparation:
prepare_data_func_for_ValidSet = functools.partial(_prepare_data_func, input_shape = input_shape, augment = False)

# Map the function to the dataset:
test_set_cifar = test_set_cifar.map(prepare_data_func_for_ValidSet, num_parallel_calls = tf.data.experimental.AUTOTUNE)

# Split the Validation set into batched samples:
test_set_cifar = test_set_cifar.batch(batch_size)

# Set to prefetch the data: improves the perforamnce.
# his allows later elements to be prepared while the current element is being processed. 
# This often improves latency and throughput, at the cost of using additional memory to store prefetched elements.
test_set_cifar= test_set_cifar.prefetch(1)


## 2.3 - Set the "steps_per_epoch" for Training and Validation Set:

From the previous code block, the dataset is compatible with Keras methods liike "model.fit()", but will also require the "steps_per_epoch" to be specified. This is the number of batches per epoch of the Dataset objects that has to be specified for the Keras method so that it can work properly together. This is done for the training batches ("steps_per_epoch") and validation batches ("validation_steps").

In [23]:
train_steps_per_epoch = math.ceil(nb_train_imgs / batch_size)

valid_steps_per_epoch = math.ceil(nb_valid_imgs / batch_size)

### The data preparation function will be placed into a ".py" utily file, so that it can be reuse later on.

## 3 - ResNet Implementation with Keras:

## Summary:

Although this is not the end of this project, it does conclude the 1st notebook relevant to the theory of these advanced deep learning models. Please go to check out __Notebook 3__ for the __Keras Applications and reusing models__.